In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from moisture_rnn import run_case

## Original Case - Single Batch

### Reproducibility Dataset

In [ ]:
reproducibility_file='../data/reproducibility_dict.pickle'

repro={}
repro.update(load_and_fix_data(reproducibility_file))
print_dict_summary(repro)

In [ ]:
param_sets_ORIG = {'id':0,
        'purpose':'reproducibility',
        'batch_size':np.inf,
        'training':None,
        'cases':['case11'],
        'scale':0,
        'rain_do':False,
#        'verbose':False,
        'verbose':1,
        'timesteps':5,
        'activation':['linear','linear'],
        'centering':[0.0,0.0],
        'hidden_units':6,
        'dense_units':1,
        'dense_layers':1,
        'DeltaE':[0,-1],    # -1.0 is to correct E bias but put at the end
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain                              
        'epochs':5000,
        'verbose_fit':0,
        'verbose_weights':True,
        'note':'check 5 should give zero error'
        }

### Using `run_case` Function

In [ ]:
reproducibility.set_seed()
print('Running reproducibility')
assert param_sets_ORIG['purpose'] == 'reproducibility'
param_sets_ORIG['initialize']=False 
case = 'case11'
case_data=repro[case]
case_data["h2"]=300
run_case(case_data,param_sets_ORIG)

### Manual Steps

In [ ]:
from moisture_rnn import create_rnn_data_1, create_rnn_data_2, train_rnn
# print(h2)
params = param_sets_ORIG
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

print(rnn_dat["x_train"].shape)
print(rnn_dat["y_train"].shape)

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True
)

In [ ]:
from moisture_rnn import rnn_predict
from utils import hash2
m = rnn_predict(model_predict, params, rnn_dat)
print(hash2(m))

In [ ]:
hours, features = rnn_dat['X'].shape

x_input=np.reshape(rnn_dat['X'],(1, hours, features))
y_output = model_predict.predict(x_input, verbose = True)

m = np.reshape(y_output,hours)

In [ ]:
print(hash2(m))
print(np.sqrt(((m - rnn_dat["Y"])**2).mean()))

In [ ]:
from data_funcs import rmse_data
print(hash2(case_data["m"]))
rmse_data(case_data)

In [ ]:
from data_funcs import rmse

In [ ]:
hours = rnn_dat['hours']
rmse(m[:hours], rnn_dat["Y"])

In [ ]:
rmse(m[:hours], case_data["fm"])

In [ ]:
np.mean(case_data["fm"] == rnn_dat["Y"])

In [ ]:
np.mean(case_data["fm"] == rnn_dat["Y"].flatten())

In [ ]:
m[:hours].shape

In [ ]:
rnn_dat["Y"].shape

In [ ]:
rnn_dat["Y"].shape

In [ ]:
rnn_dat["Y"].flatten().shape

In [ ]:
rmse(m[:hours], rnn_dat["Y"].flatten())

In [ ]:
rmse(m[:hours], rnn_dat["Y"])

In [ ]:
def rmse(a, b):
    return np.sqrt(((a - b)**2).mean())

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(m, rnn_dat["Y"]))

In [ ]:
np.sqrt(mean_squared_error(m, rnn_dat["Y"].flatten()))

In [ ]:
mse=[]
for i in range(0, len(m)):
    mse.append((m[i]-rnn_dat["Y"][i])**2)
print(np.sqrt(np.mean(mse)))

In [ ]:
m.shape

In [ ]:
rnn_dat["Y"].shape

In [ ]:
(m - rnn_dat["Y"]).shape

In [ ]:
(m - rnn_dat["Y"].flatten()).shape

In [ ]:
rnn_dat["Y"].flatten().shape

## Reproduce with Stateful Batch Training

*NOTE*: using fewer epochs as the training loss levels off much quicker with stateful.

In [ ]:
param_sets_STATEFUL = {'id':0,
        'purpose':'reproducibility',
        'batch_size':32,
        'training':5,
        'cases':['case11'],
        'scale':1,
        'rain_do':True,
#        'verbose':False,
        'verbose':1,
        'timesteps':5,
        'activation':['linear','linear'],
        'centering':[0.0,0.0],
        'hidden_units':20,
        'dense_units':1,
        'dense_layers':1,
        'DeltaE':[0,-1],    # -1.0 is to correct E bias but put at the end
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain                              
        'epochs':200,
        'verbose_fit':0,
        'verbose_weights':False,
        'note':'check 5 should give zero error',
        'verbose': False # another verbose arg used in staircase_2
        }

### Using `run_case`

In [ ]:
reproducibility.set_seed()
param_sets_STATEFUL['initialize']=False 
case = 'case11'
case_data=repro[case]
case_data['h2']=427
run_case(case_data,param_sets_STATEFUL)

### Manual Steps

In [ ]:
h2=case_data['h2']
params = param_sets_STATEFUL
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

print(rnn_dat["x_train"].shape)
print(rnn_dat["y_train"].shape)

In [ ]:
from moisture_rnn import create_RNN_2
features = rnn_dat["x_train"].shape[2]
reproducibility.set_seed() # Set seed for reproducibility
model_fit=create_RNN_2(hidden_units=params['hidden_units'], 
                    dense_units=params['dense_units'], 
                    batch_shape=rnn_dat["x_train"].shape,
                    stateful=True,
                    return_sequences=False,
                    # initial_state=h0,
                    activation=params['activation'],
                    dense_layers=params['dense_layers'],
                    verbose = False)

model_predict=create_RNN_2(hidden_units=params['hidden_units'], 
                    dense_units=params['dense_units'],  
                    input_shape=(hours,features),stateful = False,
                    return_sequences=True,
                    activation=params['activation'],
                    dense_layers=params['dense_layers'],
                    verbose = False)

In [ ]:
hash2(model_fit.get_weights())

In [ ]:
w_fitted=model_fit.get_weights()
model_predict.set_weights(w_fitted)

In [ ]:
m = rnn_predict(model_predict, params, rnn_dat)
hash2(m)

In [ ]:
# model_fit(rnn_dat['x_train'])

In [ ]:
reproducibility.set_seed()
model_predict = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True
)